In [ ]:
%load_ext autoreload
%autoreload 2

from gym.environment import MissileEnv, MissileEnvSettings
from agents.proportional_nav import ProportionalNavigationAgent
from models.missile import MissileModel
import models.physics as physics
import numpy as np
import time

from visualization.matplot_viz import MatplotVisualizer
from visualization.plotly_viz import PlotlyVisualizer


settings = MissileEnvSettings()
settings.time_step = 0.1  # Time step for the simulation
settings.realtime = False

target = MissileModel(velocity=np.array([physics.mach_to_ms(3), 0.0,  0.0]), max_acc=100 * 9.81, pos=np.array([0.0, -10_000.0, 20_000.0]))
interceptor = MissileModel(velocity=np.array([0.0, 0.0, physics.mach_to_ms(4.0)]), max_acc=100 * 9.81, pos=np.array([0.0, 0.0, 100.0]))

init_distance = np.linalg.norm(interceptor.pos - target.pos)  # Initial distance between interceptor and target

env = MissileEnv(settings=settings, target=target, interceptor=interceptor)
agent = ProportionalNavigationAgent(max_acc=100*9.81, init_distance=init_distance, n=5.0)

done = False
obs = env.reset()

viz = MatplotVisualizer()
viz.set_episode_data(env.current_episode)
viz.render(env.sim_time)

last_time = time.time()
max_render_fps = 10.0

while not done:
    action = agent.step(obs, settings.time_step)  # Get the acceleration command from the agent
    obs, reward, done, _, _ = env.step(action)  # Take a step in the environment

print ("Done")
# viz.playback(env.sim_time, speed=5.0)
viz.render(env.sim_time)
viz.save_playback("output.gif", env.sim_time, 5.0, 5)
env.close()

# Missile Interceptor Reinforcement Learning

## Proportional Guidance